# Rosenbrock

Compile, run, and plot the result from Ros.cpp


[see this paper for different methods](https://www.semanticscholar.org/paper/Improved-traditional-Rosenbrock-Wanner-methods-for-Rang/1723a6df73a7fd445d89123007e1fcaa5fc368e6#related-papers)

Also change the makefile to compare between RODASPR2 and ROS34PW2.

In [1]:
import subprocess
import sys

import os

import time

In [2]:
import numpy as np

import matplotlib
#matplotlib.use('WebAgg')
#matplotlib.use('Qt4Cairo')
#matplotlib.use('Qt5Cairo')
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt
plt.rcParams['font.family']='serif'
plt.rcParams['font.size']=10
plt.rcParams['mathtext.fontset']='stixsans'


from scipy.integrate import odeint
from scipy.integrate import Radau

In [3]:
os.chdir('..')
os.system(r'make')
os.chdir('0-test')

In [4]:
time0=time.time()

output=subprocess.check_output(["../Ros.run"]).decode(sys.stdout.encoding).split("\n")

print("time: {:10} s".format( time.time()-time0)  )

solution=np.array([ (i.split(' '))[:-1]   for i in output[:-1] ] ,np.float64)



time: 0.008903980255126953 s


In [5]:
t=solution[:,0]
y1=solution[:,1]
y2=solution[:,2]
y3=solution[:,3]


err1=solution[:,4]
err2=solution[:,5]
err3=solution[:,6]

In [6]:
def f(t,y):
    lhs=np.zeros(3)
    lhs[0]=-2*y[0]*pow(t,2) ;
    lhs[1]=2*y[0]*pow(t,2)+2*(-pow( y[1],2  )+pow( y[2],2 ) )*pow(t,1);
    lhs[2]=4*y[0]*pow(t,2)+2*(pow( y[1],2  )-pow( y[2],2 ) )*pow(t,1);
    return lhs

In [7]:
sol_py=Radau(f,0,[y1[0],y2[0],y3[0]],t[-1],atol=1e-8,rtol=1e-8)

time0=time.time()

y_py=[[y1[0],y2[0],y3[0]]]
t_py=[0]
while sol_py.status=='running' :
    sol_py.step()
    y_py.append(sol_py.y)
    t_py.append(sol_py.t)
#     print(sol_py.step_size,sol_py.t)
y_py=np.array(y_py)

print("time: {:10} s".format( time.time()-time0)  )


time: 0.1969151496887207 s


In [8]:
def g(y,t):
    return f(t,y)

In [9]:
time0=time.time()
sol_ode=odeint(g,y_py[0],t )
print("time: {:10} s".format( time.time()-time0)  )

time: 0.005058765411376953 s


<IPython.core.display.Javascript object>


In [10]:
fig=plt.figure(figsize=(9,6))
fig.subplots_adjust(bottom=0.05, left=0.1, top = 0.99, right=0.9,wspace=0.0,hspace=0.15)
fig.suptitle('')



_c=['xkcd:black','xkcd:red','xkcd:blue']
sub = fig.add_subplot(311)

sub.plot(t,y1,c=_c[0],alpha=0.5,linestyle='-',linewidth=3,label=r'$y_{1}(t)$')
sub.plot(t,y2,c=_c[1],alpha=0.5,linestyle='-',linewidth=3,label=r'$y_{2}(t)$')
sub.plot(t,y3,c=_c[2],alpha=0.5,linestyle='-',linewidth=3,label=r'$y_{3}(t)$')


sub.plot(t_py,y_py[:,0],c=_c[0],alpha=1,linestyle=':',linewidth=2,label=r'$y_{1}(t)$ scipy')
sub.plot(t_py,y_py[:,1],c=_c[1],alpha=1,linestyle=':',linewidth=2,label=r'$y_{2}(t)$ scipy')
sub.plot(t_py,y_py[:,2],c=_c[2],alpha=1,linestyle=':',linewidth=2,label=r'$y_{3}(t)$ scipy')

# sub.plot(t,sol_ode[:,0],c=_c[0],alpha=1,linestyle='--',linewidth=2,label=r'$y_{1}(t)$ scipy-odeint')
# sub.plot(t,sol_ode[:,1],c=_c[1],alpha=1,linestyle='--',linewidth=2,label=r'$y_{2}(t)$ scipy-odeint')
# sub.plot(t,sol_ode[:,2],c=_c[2],alpha=1,linestyle='--',linewidth=2,label=r'$y_{3}(t)$ scipy-odeint')


sub.legend(framealpha=0,ncol=2,loc='upper right',bbox_to_anchor=(1,.9))
sub.set_xscale('log')
# sub.set_yscale('log')

sub.set_ylabel('y')   
# sub.set_xlim(0,1)   



sub = fig.add_subplot(312)    
sub.hist(np.log10(t[1:]),color=_c[0],label=r'mine',bins=25 )
# sub.plot(t,hist)
# sub.set_xscale('log')

sub.set_ylabel('No. steps')


sub = fig.add_subplot(313)    
sub.plot(t,np.abs(err1/y1),c=_c[0],alpha=1,linestyle='--',linewidth=3,label=r'$y_{1}(t)$')
sub.plot(t,np.abs(err2/y2),c=_c[1],alpha=1,linestyle='--',linewidth=3,label=r'$y_{2}(t)$')
sub.plot(t,np.abs(err3/y3),c=_c[2],alpha=1,linestyle='--',linewidth=3,label=r'$y_{3}(t)$')
sub.legend(framealpha=0,ncol=2,loc='upper right',bbox_to_anchor=(1,.9))

sub.set_yscale('log')
sub.set_xscale('log')

sub.set_ylabel(r' $\dfrac{\Delta y}{ y} $ ')
sub.set_xlabel('t')  


plt.show()

<IPython.core.display.Javascript object>


In [11]:
fig=plt.figure(figsize=(8,4))
fig.subplots_adjust(bottom=0.15, left=0.1, top = 0.99, right=0.9,wspace=0.0,hspace=0.2)
fig.suptitle('')



_c=['xkcd:black','xkcd:red','xkcd:blue']
sub = fig.add_subplot(111)

sub.hist(np.log10(t[1:]),color=_c[0],label=r'mine',bins=25 )
sub.hist(np.log10(t_py[1:]),color=_c[2],label=r'scipy',alpha=0.5,bins=25)

# check also this
# sub.plot(t,hist,label=r'mine')
# sub.hist(t_py,label=r'scipy',alpha=0.5,bins=N)



sub.set_ylabel('No. steps')


sub.legend(framealpha=0,ncol=2,loc='upper right',bbox_to_anchor=(1,.9))





plt.show()

In [12]:
len(t)

454

In [13]:
len(t_py)

307